In [ ]:
# === Импорты ===
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score
from datasets import Dataset

# === Загрузка данных ===
train_df = pd.read_csv("data/preprocessed/clean_train.csv", sep=";")
test_df = pd.read_csv("data/preprocessed/clean_test.csv", sep=";")

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# === Токенизация ===
MODEL_NAME = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


def tokenize_function(example):
    return tokenizer(
        example["message"],
        padding="max_length",
        truncation=True,
        max_length=128
    )


train_enc = train_dataset.map(tokenize_function, batched=True)
test_enc = test_dataset.map(tokenize_function, batched=True)

train_enc.set_format("torch", columns=["input_ids", "attention_mask", "is_toxic"])
test_enc.set_format("torch", columns=["input_ids", "attention_mask", "is_toxic"])

# === Модель и устройство ===
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

# === DataLoader и оптимизатор ===
train_loader = DataLoader(train_enc, batch_size=8, shuffle=True)
test_loader = DataLoader(test_enc, batch_size=8)

optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# === Обучение ===
epochs = 3
model.train()

for epoch in range(epochs):
    total_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch + 1}")
    for batch in loop:
        optimizer.zero_grad()
        inputs = {k: v.to(device) for k, v in batch.items() if k != "is_toxic"}
        labels = batch["is_toxic"].to(device)

        outputs = model(**inputs)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"Средняя потеря за эпоху {epoch + 1}: {avg_loss:.4f}")

# === Оценка ===
model.eval()
preds, labels = [], []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Оценка"):
        inputs = {k: v.to(device) for k, v in batch.items() if k != "is_toxic"}
        y_true = batch["is_toxic"].to(device)

        outputs = model(**inputs)
        y_pred = torch.argmax(outputs.logits, dim=1)

        preds.extend(y_pred.cpu().numpy())
        labels.extend(y_true.cpu().numpy())

print("\n=== Отчёт по метрикам ===")
print(classification_report(labels, preds, digits=3))
print("F1-score:", round(f1_score(labels, preds), 3))

# === Сохранение модели ===
model.save_pretrained("models/roberta-toxic")
tokenizer.save_pretrained("models/roberta-toxic")

print("\nМодель и токенизатор сохранены в 'models/roberta-toxic'")

W1016 18:21:21.558000 95505 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


Map:   0%|          | 0/12711 [00:00<?, ? examples/s]

Map:   0%|          | 0/228 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1:  37%|███▋      | 581/1589 [03:28<06:45,  2.48it/s, loss=0.0406]